# Data Science Job Change Prediction
- Author: Le Liu
- Course: COMP3010J Machine Learning


## 1. Project Overview

This project aims to predict whether a candidate is looking for a job change based on various demographic and professional features. Then infer the key factors influencing their decision.

**Dataset:** `data-science-job-change.csv`

**Problem Type:** Binary Classification

**Target Variable:** `target` (1.0 = Looking for job change, 0.0 = Not looking for job change)



*Project Structure*

1. **Introduction** - Project overview and objectives
2. **Load and Analyse Data** - Data loading and initial exploration
3. **Data Cleaning** - Handle missing values and data quality issues
4. **Data Visualisation** - Exploratory data analysis with plots
5. **Attribute Selection** - Feature selection and engineering
6. **Model Selection and Experiments** - Train and compare models
7. **Final Model Training** - Train the best model
8. **Further Analysis and Discussion** - Model interpretation
9. **Discussion** - Conclusions and future work

---
## 2. Data Loading & Analysis

In [ ]:
# Load the dataset
import pandas as pd


df = pd.read_csv('data-science-job-change.csv')

# Display basic information
print(f"Dataset shape: {df.shape}")
print(f"Number of rows: {df.shape[0]}")
print(f"Number of columns: {df.shape[1]}")
print("\nFirst 5 rows of the dataset:")
df.head()

In [ ]:
# Display statistical summary
print("Statistical Summary:")
print("="*60)
numerical_cols = ['city_development_index', 'training_hours', 'target']
df[numerical_cols].describe()

In [ ]:
# Check missing values
missing_data = pd.DataFrame({
    'Column': df.columns,
    'Missing_Count': df.isnull().sum(),
    'Missing_Percentage': (df.isnull().sum() / len(df) * 100).round(3),
    'Data_Type': df.dtypes,
    'Unique_Values': [df[col].nunique() for col in df.columns]
})

print("Data Health Report:")
print("="*80)
print(missing_data.to_string(index=False))

### 2.1 Target Variable Analysis (Class Balance Check)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Set visualization style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

# Target variable analysis
print("=" * 80)
print("TARGET VARIABLE ANALYSIS: 'target' (Job Change Intention)")
print("=" * 80)

target_counts = df['target'].value_counts().sort_index()
target_pct = df['target'].value_counts(normalize=True).sort_index() * 100

print("\nClass Distribution:")
print("-" * 50)
for val, count, pct in zip(target_counts.index, target_counts.values, target_pct.values):
    label = "Not Looking for Change" if val == 0.0 else "Looking for Change"
    print(f"  Class {int(val)} ({label:25s}): {count:>6,} ({pct:>5.2f}%)")

# Calculate imbalance ratio
imbalance_ratio = target_counts.max() / target_counts.min()
print(f"\nImbalance Ratio: {imbalance_ratio:.2f}:1")


# Visualization
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Bar plot
axes[0].bar(target_counts.index, target_counts.values, color=['#3498db', '#e74c3c'], alpha=0.7)
axes[0].set_xlabel('Target Class', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Count', fontsize=12, fontweight='bold')
axes[0].set_title('Target Variable Distribution', fontsize=14, fontweight='bold')
axes[0].set_xticks([0, 1])
axes[0].set_xticklabels(['Not Looking\nfor Change (0)', 'Looking for\nChange (1)'])
for i, (val, count) in enumerate(zip(target_counts.index, target_counts.values)):
    axes[0].text(i, count, f'{count:,}\n({target_pct.values[i]:.1f}%)', 
                 ha='center', va='bottom', fontweight='bold')

# Pie chart
colors = ['#3498db', '#e74c3c']
axes[1].pie(target_counts.values, labels=['Not Looking (0)', 'Looking (1)'], 
           autopct='%1.1f%%', colors=colors, startangle=90, textprops={'fontsize': 12, 'fontweight': 'bold'})
axes[1].set_title('Target Class Proportion', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

print("\n" + "=" * 80)

### 2.2 Numerical Features Analysis

In [ ]:
from scipy import stats

# Select numerical features
numerical_features = ['city_development_index', 'training_hours']

print("=" * 80)
print("NUMERICAL FEATURES ANALYSIS")
print("=" * 80)

# Detailed statistics for each numerical feature
for col in numerical_features:
    print(f"\nFeature: '{col}'")
    print("-" * 60)
    
    data = df[col].dropna()
    
    # Basic statistics
    print(f"   Count:    {len(data):>10,}")
    print(f"   Mean:     {data.mean():>10.4f}")
    print(f"   Median:   {data.median():>10.4f}")
    print(f"   Std Dev:  {data.std():>10.4f}")
    print(f"   Min:      {data.min():>10.4f}")
    print(f"   Max:      {data.max():>10.4f}")
    print(f"   Range:    {data.max() - data.min():>10.4f}")
    print(f"   Value Range: [{data.min()}, {data.max()}]")
    
    # Distribution shape
    skewness = stats.skew(data)
    kurtosis = stats.kurtosis(data)
    print(f"\n   Skewness: {skewness:>10.4f}  ", end="")
    if abs(skewness) < 0.5:
        print("(Nearly symmetric)")
    elif skewness > 0:
        print("(Right-skewed / Positive skew)")
    else:
        print("Left-skewed / Negative skew)")
    
    print(f"   Kurtosis: {kurtosis:>10.4f}  ", end="")
    if abs(kurtosis) < 1:
        print("(Normal tail)")
    elif kurtosis > 0:
        print("(Heavy-tailed / Outliers present)")
    else:
        print("(Light-tailed)")
    
    # Outlier detection (IQR method)
    Q1 = data.quantile(0.25)
    Q3 = data.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = data[(data < lower_bound) | (data > upper_bound)]
    
    print(f"\n   Outliers: {len(outliers):>10,} ({len(outliers)/len(data)*100:.2f}%)")
    print(f"   IQR Range: [{lower_bound:.4f}, {upper_bound:.4f}]")

print("\n" + "=" * 80)

# Visualization
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('Numerical Features Distribution Analysis', fontsize=16, fontweight='bold', y=1.00)

for idx, col in enumerate(numerical_features):
    data = df[col].dropna()
    
    # Histogram with KDE
    axes[idx, 0].hist(data, bins=50, edgecolor='black', alpha=0.7, color='steelblue', density=True)
    data.plot(kind='kde', ax=axes[idx, 0], color='red', linewidth=2)
    axes[idx, 0].set_title(f'{col} - Distribution', fontweight='bold')
    axes[idx, 0].set_xlabel(col)
    axes[idx, 0].set_ylabel('Density')
    axes[idx, 0].axvline(data.mean(), color='green', linestyle='--', linewidth=2, label=f'Mean: {data.mean():.2f}')
    axes[idx, 0].axvline(data.median(), color='orange', linestyle='--', linewidth=2, label=f'Median: {data.median():.2f}')
    axes[idx, 0].legend()
    
    # Box plot
    axes[idx, 1].boxplot(data, vert=False, patch_artist=True,
                        boxprops=dict(facecolor='lightblue', color='blue'),
                        medianprops=dict(color='red', linewidth=2),
                        whiskerprops=dict(color='blue'),
                        capprops=dict(color='blue'))
    axes[idx, 1].set_title(f'{col} - Box Plot (Outlier Detection)', fontweight='bold')
    axes[idx, 1].set_xlabel(col)
    axes[idx, 1].grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

### 2.3 Categorical Features

In [ ]:
# Categorical features
categorical_features = ['city', 'gender', 'relevent_experience', 'enrolled_university', 
                        'education_level', 'major_discipline', 'experience', 
                        'company_size', 'company_type', 'last_new_job']

print("=" * 80)
print("CATEGORICAL FEATURES - UNIQUE VALUES OVERVIEW")
print("=" * 80)

cat_summary = []
for col in categorical_features:
    n_unique = df[col].nunique()
    n_missing = df[col].isnull().sum()
    missing_pct = (n_missing / len(df)) * 100
    
    cat_summary.append({
        'Feature': col,
        'Unique_Values': n_unique,
        'Missing_Count': n_missing,
        'Missing_%': f"{missing_pct:.1f}%"
    })

cat_df = pd.DataFrame(cat_summary)
print("\n" + cat_df.to_string(index=False))

# Detailed display for features with reasonable number of categories (exclude 'city')
print("\n" + "=" * 80)
print("DETAILED VALUES FOR EACH FEATURE (excluding high-cardinality features)")
print("=" * 80)

for col in categorical_features:
    n_unique = df[col].nunique()
    
    # Skip features with too many unique values (like 'city' with 123 values)
    if n_unique > 25:
        print(f"\n Feature: '{col}'")
        print(f"   Unique Values: {n_unique} (too many to display)")
        print(f"   Note: High cardinality feature - will use Target Encoding")
        continue
    
    # Display all unique values for features with reasonable cardinality
    print(f"Feature: '{col}'")
    print(f"   Unique Values: {n_unique}")
    
    # Get value counts including missing
    value_counts = df[col].value_counts(dropna=False)
    
    print(f"   All Values: ", end="")
    all_values = df[col].dropna().unique().tolist()
    
    if n_unique <= 10:
        # For features with ≤10 values, display vertically with counts
        print()
        for val in sorted([str(v) for v in all_values]):
            count = value_counts.get(val, 0)
            pct = (count / len(df)) * 100
            print(f"      • {val:30s}  ({count:>6,} samples, {pct:>5.2f}%)")
        
        # Show missing values if any
        missing_count = df[col].isnull().sum()
        if missing_count > 0:
            print(f"      • [MISSING]                       ({missing_count:>6,} samples, {missing_count/len(df)*100:>5.2f}%)")
    else:
        # For features with 11-25 values, display horizontally 
        print(f"{sorted([str(v) for v in all_values])}")

print("\n" + "=" * 80)

### 2.4 High-Priority Categorical Features - Detailed Analysis

In [ ]:
# Analyze high-priority categorical features in detail
high_priority_features = ['gender', 'company_size', 'company_type', 'major_discipline', 'experience']

print("=" * 80)
print("HIGH-PRIORITY CATEGORICAL FEATURES - DETAILED FREQUENCY ANALYSIS")
print("=" * 80)

for col in high_priority_features:
    print(f"\n{'=' * 80}")
    print(f"Feature: '{col}'")
    print(f"{'=' * 80}")
    
    # Value counts
    value_counts = df[col].value_counts(dropna=False)
    value_pcts = df[col].value_counts(normalize=True, dropna=False) * 100
    
    # Create summary table
    summary_df = pd.DataFrame({
        'Value': value_counts.index,
        'Count': value_counts.values,
        'Percentage': [f"{pct:.2f}%" for pct in value_pcts.values]
    })
    
    print(f"\n   Total Unique Values: {df[col].nunique()}")
    print(f"   Missing Values: {df[col].isnull().sum()} ({df[col].isnull().sum()/len(df)*100:.2f}%)")
    print(f"\n   Frequency Distribution:")
    print("   " + "-" * 70)
    
    # Display top 15 values (or all if less than 15)
    display_limit = min(15, len(summary_df))
    for idx, row in summary_df.head(display_limit).iterrows():
        val_str = str(row['Value']) if pd.notna(row['Value']) else '**MISSING**'
        print(f"   {val_str:30s} {row['Count']:>8,}   ({row['Percentage']:>7s})")
    
    if len(summary_df) > display_limit:
        print(f"   ... ({len(summary_df) - display_limit} more values)")
    
    # Identify rare categories (< 1% of data)
    rare_cats = value_pcts[value_pcts < 1.0]
    if len(rare_cats) > 0:
        print(f"\n   ⚠️  Rare categories (< 1%): {len(rare_cats)} values")
        print(f"      Consider combining or special handling")

print("\n" + "=" * 80)

### 2.5 Feature vs Target - Relationship Analysis

In [ ]:
# Analyze key features' relationship with target variable
key_features = ['relevent_experience', 'company_size', 'company_type', 'education_level']

print("=" * 80)
print("FEATURE vs TARGET RELATIONSHIP ANALYSIS")
print("=" * 80)

for col in key_features:
    print(f"\n{'=' * 80}")
    print(f"Feature: '{col}' vs Target (Job Change Intention)")
    print(f"{'=' * 80}")
    
    # Create crosstab without margins first
    crosstab = pd.crosstab(df[col], df['target'], dropna=False)
    
    # Calculate percentages (target=1 rate for each category)
    target_rate = pd.crosstab(df[col], df['target'], normalize='index', dropna=False)
    
    # Combine into summary
    summary_data = []
    for cat in crosstab.index:
        total = crosstab.loc[cat].sum()
        target_0 = crosstab.loc[cat, 0.0] if 0.0 in crosstab.columns else 0
        target_1 = crosstab.loc[cat, 1.0] if 1.0 in crosstab.columns else 0
        rate_1 = target_rate.loc[cat, 1.0] * 100 if 1.0 in target_rate.columns else 0
        
        summary_data.append({
            'Category': cat,
            'Total_Count': int(total),
            'Target=0': int(target_0),
            'Target=1': int(target_1),
            'Target=1_Rate': f"{rate_1:.2f}%"
        })
    
    summary = pd.DataFrame(summary_data)
    print("\n" + summary.to_string(index=False))
    
    # Highlight significant differences
    rates = [float(row['Target=1_Rate'].rstrip('%')) for row in summary_data]
    avg_rate = np.mean(rates)
    print(f"\n   Average Target=1 Rate: {avg_rate:.2f}%")
    
    high_rate = [row['Category'] for row, rate in zip(summary_data, rates) if rate > avg_rate + 5]
    low_rate = [row['Category'] for row, rate in zip(summary_data, rates) if rate < avg_rate - 5]
    
    if high_rate:
        print(f"Higher likelihood of job change: {high_rate}")
    if low_rate:
        print(f"Lower likelihood of job change: {low_rate}")

print("\n" + "=" * 80)

### 2.6 Data Quality Checks

In [ ]:
print("=" * 80)
print("DATA QUALITY CHECKS")
print("=" * 80)

# 1. Duplicate rows check
duplicate_count = df.duplicated().sum()
print(f"Duplicate Rows: {duplicate_count}")
if duplicate_count > 0:
    print(f"Found {duplicate_count} duplicate rows")
    print(f"Action: Review and consider removing duplicates")
else:
    print("No duplicate rows found")

# 2. Duplicate enrollee_id check
duplicate_ids = df['enrollee_id'].duplicated().sum()
print(f"Duplicate Enrollee IDs: {duplicate_ids}")
if duplicate_ids > 0:
    print(f"Found {duplicate_ids} duplicate IDs - possible data entry errors")
else:
    print("All enrollee IDs are unique")
print("\n" + "=" * 80)

### 2.8 Summary of Data Analysis Findings

**Key Insights from Exploratory Data Analysis:**

**Target Variable**
- Check class balance and imbalance ratio
- Determine if sampling/weighting strategies are needed

**Numerical Features (2 features)**
- **Distribution shape**: Skewness & Kurtosis analysis
- **Outlier detection**: IQR method for anomaly identification
- **Statistical summary**: Mean, median, std, range

**Categorical Features (10 features)**
- **Unique values count**: Understanding cardinality
- **Missing data**: Identify features requiring imputation
- **Frequency distribution**: Find rare/dominant categories
- **Feature-Target relationship**: Identify predictive patterns

**Data Quality**
- Duplicate records check
- ID uniqueness validation
- Format consistency (e.g., `company_size` "10/49" issue)
- Range validation for numerical features

---

**Next Steps:** Based on findings, proceed to Data Visualisation and then Data Cleaning (Section 3)

### 2.9 Key Data Visualisations

In this section, we visualize the relationships between key features and the target variable to gain insights into what factors influence job change decisions.

**Visualisation Strategy:**
1. **Categorical Features vs Target** - Compare job change rates across categories
2. **Numerical Features Distribution** - Compare distributions between two target classes
3. **Key Insights** - Summarize findings from visualisations

In [ ]:
# Prepare data for visualization
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Set style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (16, 12)

print("="*80)
print("DATA VISUALISATION: Feature Relationships with Target Variable")
print("="*80)

#### Visualisation 1: Key Categorical Features vs Job Change Rate

In [ ]:
# Select key categorical features for visualization
key_cat_features = ['relevent_experience', 'company_size', 'company_type', 'education_level']

fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Job Change Rate by Key Categorical Features', fontsize=16, fontweight='bold', y=0.995)

for idx, feature in enumerate(key_cat_features):
    row = idx // 2
    col = idx % 2
    ax = axes[row, col]
    
    # Calculate target rate for each category
    crosstab = pd.crosstab(df[feature], df['target'])
    target_rate = (crosstab[1.0] / crosstab.sum(axis=1) * 100).sort_values(ascending=False)
    
    # Create bar plot
    colors = plt.cm.RdYlGn_r(target_rate / 100)
    bars = ax.barh(range(len(target_rate)), target_rate.values, color=colors, edgecolor='black', linewidth=1.2)
    
    # Customize plot
    ax.set_yticks(range(len(target_rate)))
    ax.set_yticklabels(target_rate.index, fontsize=10)
    ax.set_xlabel('Job Change Rate (%)', fontsize=11, fontweight='bold')
    ax.set_title(f'{feature}', fontsize=12, fontweight='bold')
    ax.axvline(df['target'].mean() * 100, color='red', linestyle='--', linewidth=2, label='Overall Average')
    
    # Add value labels
    for i, (v, count) in enumerate(zip(target_rate.values, crosstab.sum(axis=1)[target_rate.index])):
        ax.text(v + 1, i, f'{v:.1f}% (n={count:,})', va='center', fontsize=9)
    
    ax.legend(loc='lower right')
    ax.set_xlim(0, max(target_rate.values) * 1.15)

plt.tight_layout()
plt.show()

print("\nKey Observations:")
print("-" * 80)
print("1. Relevant Experience: Candidates WITH relevant experience have LOWER job change rates")
print("2. Company Size: Smaller companies (<10) show higher job change tendency")
print("3. Company Type: Pvt Ltd companies have highest retention (lowest change rate)")
print("4. Education Level: Graduate level shows balanced job change behavior")

#### Visualisation 2: Experience Level Distribution by Target

In [ ]:
# Visualize experience distribution for both target classes
fig, axes = plt.subplots(1, 2, figsize=(16, 6))
fig.suptitle('Experience Distribution: Job Changers vs Non-Changers', fontsize=16, fontweight='bold')

# Convert experience to numeric for plotting
exp_mapping = {
    '<1': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5,
    '6': 6, '7': 7, '8': 8, '9': 9, '10': 10,
    '11': 11, '12': 12, '13': 13, '14': 14, '15': 15,
    '16': 16, '17': 17, '18': 18, '19': 19, '20': 20, '>20': 21
}

df_temp = df.copy()
df_temp['experience_numeric'] = df_temp['experience'].map(exp_mapping)

# Plot 1: Histogram comparison
for target_val, color, label in [(0, '#3498db', 'Not Looking for Change'), 
                                   (1, '#e74c3c', 'Looking for Change')]:
    data = df_temp[df_temp['target'] == target_val]['experience_numeric'].dropna()
    axes[0].hist(data, bins=22, alpha=0.6, label=label, color=color, edgecolor='black')

axes[0].set_xlabel('Years of Experience', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Count', fontsize=12, fontweight='bold')
axes[0].set_title('Experience Distribution Comparison', fontsize=13, fontweight='bold')
axes[0].legend()
axes[0].grid(axis='y', alpha=0.3)

# Plot 2: Box plot comparison
df_temp.boxplot(column='experience_numeric', by='target', ax=axes[1], patch_artist=True)
axes[1].set_xlabel('Target (0=Not Looking, 1=Looking)', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Years of Experience', fontsize=12, fontweight='bold')
axes[1].set_title('Experience Distribution by Target', fontsize=13, fontweight='bold')
plt.suptitle('')  # Remove default title

plt.tight_layout()
plt.show()

print("\nKey Observations:")
print("-" * 80)
print(f"Average experience (Not looking for change): {df_temp[df_temp['target']==0]['experience_numeric'].mean():.2f} years")
print(f"Average experience (Looking for change):     {df_temp[df_temp['target']==1]['experience_numeric'].mean():.2f} years")
print("Job changers tend to have slightly LOWER average experience")

#### Visualisation 3: Training Hours and City Development Index

In [ ]:
# Visualize numerical features
fig, axes = plt.subplots(1, 2, figsize=(16, 6))
fig.suptitle('Numerical Features: Training Hours & City Development Index', fontsize=16, fontweight='bold')

# Plot 1: Training Hours by Target
for target_val, color, label in [(0, '#3498db', 'Not Looking (0)'), 
                                   (1, '#e74c3c', 'Looking (1)')]:
    data = df[df['target'] == target_val]['training_hours']
    axes[0].hist(data, bins=30, alpha=0.6, label=label, color=color, edgecolor='black')

axes[0].set_xlabel('Training Hours', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Count', fontsize=12, fontweight='bold')
axes[0].set_title('Training Hours Distribution by Target', fontsize=13, fontweight='bold')
axes[0].legend()
axes[0].axvline(df[df['target']==0]['training_hours'].mean(), color='#3498db', linestyle='--', linewidth=2, label='Mean (0)')
axes[0].axvline(df[df['target']==1]['training_hours'].mean(), color='#e74c3c', linestyle='--', linewidth=2, label='Mean (1)')
axes[0].grid(axis='y', alpha=0.3)

# Plot 2: City Development Index by Target
df.boxplot(column='city_development_index', by='target', ax=axes[1], patch_artist=True)
axes[1].set_xlabel('Target (0=Not Looking, 1=Looking)', fontsize=12, fontweight='bold')
axes[1].set_ylabel('City Development Index', fontsize=12, fontweight='bold')
axes[1].set_title('City Development Index by Target', fontsize=13, fontweight='bold')
plt.suptitle('')

plt.tight_layout()
plt.show()

print("\nKey Observations:")
print("-" * 80)
print(f"Average training hours (Not looking): {df[df['target']==0]['training_hours'].mean():.1f} hours")
print(f"Average training hours (Looking):     {df[df['target']==1]['training_hours'].mean():.1f} hours")
print(f"Average CDI (Not looking): {df[df['target']==0]['city_development_index'].mean():.3f}")
print(f"Average CDI (Looking):     {df[df['target']==1]['city_development_index'].mean():.3f}")
print("\nCandidates in MORE developed cities are LESS likely to change jobs")

#### Summary of Visualisation Insights

**Key Findings:**

1. **Relevant Experience Impact**
   - Candidates WITHOUT relevant experience are MORE likely to seek job changes
   - This suggests career dissatisfaction or seeking better fit

2. **Company Size Effect**
   - Smaller companies (<10 employees) have higher turnover rates
   - Larger companies (1000+) show better retention

3. **Experience Level Paradox**
   - Job changers have slightly LOWER average experience
   - Mid-career professionals (5-10 years) show highest mobility

4. **City Development Factor**
   - Candidates in less developed cities are MORE likely to change jobs
   - Economic opportunities may drive this pattern

5. **Training Hours**
   - Similar training hours across both groups
   - Training alone doesn't predict job change behavior

**Implications for Modeling:**
- `relevent_experience`, `company_size`, and `city_development_index` are likely strong predictors
- `experience` shows non-linear relationship (requires careful feature engineering)
- `training_hours` may be less predictive (but still worth including)

---

---

##  3. Data Cleaning & Feature Engineering Strategy

Based on the comprehensive data analysis above, here is the detailed processing strategy for each of the 14 features:

###  3.1 Strategy Overview Table

| # | Feature | Type | Action | Priority | Reason |
|---|---------|------|--------|----------|--------|
| 1 | `enrollee_id` | ID | **DELETE** | 🔴 High | No predictive value - just an identifier |
| 2 | `city` | Categorical (123 values) | **Target Encoding** | 🟡 Medium | Too many categories for One-Hot |
| 3 | `city_development_index` | Numerical | **Keep as-is** | 🟢 Low | Already numeric, no missing values |
| 4 | `gender` | Categorical (3 values) | **Fill Missing + One-Hot** | 🟡 Medium | 23% missing, create "Unknown" category |
| 5 | `relevent_experience` | Binary | **Label Encoding** | 🟢 Low | No missing, convert to 0/1 |
| 6 | `enrolled_university` | Categorical (3 values) | **Fill Missing + One-Hot** | 🟢 Low | 2% missing, 3 clear categories |
| 7 | `education_level` | Ordinal (5 values) | **Ordinal Encoding** | 🟡 Medium | 2% missing, has natural order |
| 8 | `major_discipline` | Categorical (6 values) | **Fill Missing + One-Hot** | 🟡 Medium | 15% missing, may correlate with education |
| 9 | `experience` | Ordinal (22 values) | **Ordinal Encoding** | 🔴 High | Convert to numeric years (e.g., "<1"→0, ">20"→21) |
| 10 | `company_size` | Ordinal (8 values) | **Fix Format + Ordinal** | 🔴 High | 31% missing, FIX "10/49" → "10-49" |
| 11 | `company_type` | Categorical (6 values) | **Fill Missing + One-Hot** | 🟡 Medium | 32% missing, create "Unknown" |
| 12 | `last_new_job` | Ordinal (6 values) | **Ordinal Encoding** | 🟢 Low | 2% missing, convert to numeric |
| 13 | `training_hours` | Numerical | **Keep as-is** | 🟢 Low | Already numeric, no missing values |
| 14 | `target` | Binary | **Keep as-is** | N/A | Target variable - no transformation needed |

---

### 3.2 Data Clean and Base Encoding


In [ ]:
# Create a copy of the original dataframe for cleaning
df_clean = df.copy()

print(f"Original dataset shape: {df_clean.shape}")
print(f"Starting data cleaning process...\n")

#### 1. `enrollee_id`
Delete the column `enrollee_id` because it is not interesting for our analysis.

In [ ]:
# Feature 1: enrollee_id
print("="*70)
print("Processing Feature 1: enrollee_id")
print("="*70)

missing_count = df_clean['enrollee_id'].isna().sum()
print(f"\nMissing before: {missing_count} ({missing_count/len(df_clean)*100:.2f}%)")

# Delete the column (no predictive value)
df_clean = df_clean.drop('enrollee_id', axis=1)

print(f"\nFeature 1 processed: enrollee_id")
print(f"   Action: Deleted (identifier column)")
print(f"   New shape: {df_clean.shape}")
print(f"   Columns remaining: {df_clean.shape[1]}\n")

#### 2. `company_size`
**Action:** Fix format error ('10/49'→'10-49'), create missing indicator, fill with median, ordinal encoding (0-7)  
**Reason:** 31% missing + format issue; ordinal relationship exists (company scale)

In [ ]:
# Feature 2: company_size
print("="*70)
print("Processing Feature 2: company_size")
print("="*70)

missing_count = df_clean['company_size'].isna().sum()
print(f"\nMissing before: {missing_count} ({missing_count/len(df_clean)*100:.2f}%)")

# Step 1: Fix formatting error
print("\nStep 1: Fix formatting error '10/49' -> '10-49'")
df_clean['company_size'] = df_clean['company_size'].replace('10/49', '10-49')
print(f"   Fixed {df_clean['company_size'].value_counts().get('10-49', 0):,} instances")

# Step 2: Create missing indicator
print("\nStep 2: Create missing indicator feature")
df_clean['company_size_missing'] = df_clean['company_size'].isna().astype(int)
print(f"   Created 'company_size_missing' (1=missing, 0=known)")

# Step 3: Fill missing with median category
print("\nStep 3: Fill missing with median category")
median_category = '50-99'
df_clean['company_size'] = df_clean['company_size'].fillna(median_category)
print(f"   Filled {missing_count:,} missing values with '{median_category}'")
print(f"   Missing after: {df_clean['company_size'].isna().sum()}")

# Step 4: Ordinal encoding
print("\nStep 4: Ordinal encoding (small to large)")
size_order = {
    '<10': 0, '10-49': 1, '50-99': 2, '100-500': 3,
    '500-999': 4, '1000-4999': 5, '5000-9999': 6, '10000+': 7
}
df_clean['company_size'] = df_clean['company_size'].map(size_order)

print("   Encoding mapping:")
for key, value in size_order.items():
    count = (df_clean['company_size'] == value).sum()
    print(f"      {key:12s} -> {value}  ({count:,} samples)")

print(f"\nFeature 2 processed: company_size + company_size_missing")
print(f"   company_size: Type={df_clean['company_size'].dtype}, Range=[{df_clean['company_size'].min()}, {df_clean['company_size'].max()}]")
print(f"   company_size_missing: {df_clean['company_size_missing'].value_counts().to_dict()}")
print(f"   Shape: {df_clean.shape}\n")

#### 3. `company_type`
**Action:** Fill missing with 'Unknown', One-Hot encoding  
**Reason:** 32% missing; 6 nominal categories (no natural order)

In [ ]:
# Feature 3: company_type
print("="*70)
print("Processing Feature 3: company_type")
print("="*70)

missing_count = df_clean['company_type'].isna().sum()
print(f"\nMissing before: {missing_count} ({missing_count/len(df_clean)*100:.2f}%)")

# Step 1: Fill missing values
print("\nStep 1: Fill missing values with 'Unknown'")
df_clean['company_type'] = df_clean['company_type'].fillna('Unknown')
print(f"   Filled {missing_count:,} missing values with 'Unknown'")
print(f"   Missing after: {df_clean['company_type'].isna().sum()}")

# Step 2: One-Hot encoding
print("\nStep 2: One-Hot encoding")
print(f"   Categories: {sorted(df_clean['company_type'].unique())}")

company_type_dummies = pd.get_dummies(df_clean['company_type'], prefix='company_type', drop_first=False)
df_clean = pd.concat([df_clean, company_type_dummies], axis=1)
df_clean = df_clean.drop('company_type', axis=1)

print(f"   Created {len(company_type_dummies.columns)} binary columns:")
for col in sorted(company_type_dummies.columns):
    count = df_clean[col].sum()
    print(f"      {col:35s}: {count:,} samples")

print(f"\nFeature 3 processed: company_type")
print(f"   New columns added: {len(company_type_dummies.columns)}")
print(f"   Shape: {df_clean.shape}\n")

#### 4. `gender`
**Action:** Fill missing with 'Unknown', One-Hot encoding  
**Reason:** 23% missing; 3 nominal categories (Male/Female/Other)

In [ ]:
# Feature 4: gender
print("="*70)
print("Processing Feature 4: gender")
print("="*70)

missing_count = df_clean['gender'].isna().sum()
print(f"\nMissing before: {missing_count} ({missing_count/len(df_clean)*100:.2f}%)")

# Step 1: Fill missing values
print("\nStep 1: Fill missing values with 'Unknown'")
df_clean['gender'] = df_clean['gender'].fillna('Unknown')
print(f"   Filled {missing_count:,} missing values with 'Unknown'")
print(f"   Missing after: {df_clean['gender'].isna().sum()}")

# Step 2: One-Hot encoding
print("\nStep 2: One-Hot encoding")
gender_dummies = pd.get_dummies(df_clean['gender'], prefix='gender', drop_first=False)
df_clean = pd.concat([df_clean, gender_dummies], axis=1)
df_clean = df_clean.drop('gender', axis=1)

print(f"   Created {len(gender_dummies.columns)} binary columns:")
for col in sorted(gender_dummies.columns):
    count = df_clean[col].sum()
    print(f"      {col:35s}: {count:,} samples")

print(f"\nFeature 4 processed: gender")
print(f"   Shape: {df_clean.shape}\n")

#### 5. `major_discipline`
**Action:** Fill missing with 'Unknown', One-Hot encoding  
**Reason:** 15% missing; 6 nominal categories (STEM, Business, etc.)

In [ ]:
# Feature 5: major_discipline
print("="*70)
print("Processing Feature 5: major_discipline")
print("="*70)

missing_count = df_clean['major_discipline'].isna().sum()
print(f"\nMissing before: {missing_count} ({missing_count/len(df_clean)*100:.2f}%)")

# Step 1: Fill missing values
print("\nStep 1: Fill missing values with 'Unknown'")
df_clean['major_discipline'] = df_clean['major_discipline'].fillna('Unknown')
print(f"   Filled {missing_count:,} missing values with 'Unknown'")
print(f"   Missing after: {df_clean['major_discipline'].isna().sum()}")

# Step 2: One-Hot encoding
print("\nStep 2: One-Hot encoding")
major_dummies = pd.get_dummies(df_clean['major_discipline'], prefix='major', drop_first=False)
df_clean = pd.concat([df_clean, major_dummies], axis=1)
df_clean = df_clean.drop('major_discipline', axis=1)

print(f"   Created {len(major_dummies.columns)} binary columns:")
for col in sorted(major_dummies.columns):
    count = df_clean[col].sum()
    print(f"      {col:35s}: {count:,} samples")

print(f"\nFeature 5 processed: major_discipline")
print(f"   Shape: {df_clean.shape}\n")

#### 6. `experience`
**Action:** Fill missing with median, ordinal encoding ('<1'→0, '20'→20, '>20'→21)  
**Reason:** Few missing values; clear ordinal relationship (years of experience)

In [ ]:
# Feature 6: experience
print("="*70)
print("Processing Feature 6: experience")
print("="*70)

missing_count = df_clean['experience'].isna().sum()
print(f"\nMissing before: {missing_count} ({missing_count/len(df_clean)*100:.2f}%)")

# Define ordinal mapping
exp_mapping = {
    '<1': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5,
    '6': 6, '7': 7, '8': 8, '9': 9, '10': 10,
    '11': 11, '12': 12, '13': 13, '14': 14, '15': 15,
    '16': 16, '17': 17, '18': 18, '19': 19, '20': 20, '>20': 21
}

# Step 1: Fill missing with median
print("\nStep 1: Fill missing with median")
temp_numeric = df_clean['experience'].dropna().map(exp_mapping)
median_value = temp_numeric.median()
median_key = min(exp_mapping.items(), key=lambda x: abs(x[1] - median_value))[0]
df_clean['experience'] = df_clean['experience'].fillna(median_key)
print(f"   Filled {missing_count:,} missing values with median '{median_key}' (value={exp_mapping[median_key]})")

# Step 2: Ordinal encoding
print("\nStep 2: Ordinal encoding")
df_clean['experience'] = df_clean['experience'].map(exp_mapping)
print(f"   Encoded: '<1'->0, '1'->1, ..., '>20'->21")

print(f"\nFeature 6 processed: experience")
print(f"   Type: {df_clean['experience'].dtype}, Range: [{df_clean['experience'].min()}, {df_clean['experience'].max()}] years")
print(f"   Missing after: {df_clean['experience'].isna().sum()}")
print(f"   Shape: {df_clean.shape}\n")

#### 7. `education_level`
**Action:** Fill missing with mode, ordinal encoding (Primary→1 to PhD→5)  
**Reason:** 2% missing; clear educational hierarchy

In [ ]:
# Feature 7: education_level
print("="*70)
print("Processing Feature 7: education_level")
print("="*70)

missing_count = df_clean['education_level'].isna().sum()
print(f"\nMissing before: {missing_count} ({missing_count/len(df_clean)*100:.2f}%)")

# Step 1: Fill missing with mode
print("\nStep 1: Fill missing with mode")
mode_value = df_clean['education_level'].mode()[0]
df_clean['education_level'] = df_clean['education_level'].fillna(mode_value)
print(f"   Filled {missing_count:,} missing values with mode '{mode_value}'")

# Step 2: Ordinal encoding
print("\nStep 2: Ordinal encoding (educational hierarchy)")
edu_mapping = {
    'Primary School': 1,
    'High School': 2,
    'Graduate': 3,
    'Masters': 4,
    'Phd': 5
}
df_clean['education_level'] = df_clean['education_level'].map(edu_mapping)
print(f"   Encoded: Primary School->1, High School->2, Graduate->3, Masters->4, PhD->5")

print(f"\nFeature 7 processed: education_level")
print(f"   Type: {df_clean['education_level'].dtype}, Range: [{df_clean['education_level'].min()}, {df_clean['education_level'].max()}]")
print(f"   Missing after: {df_clean['education_level'].isna().sum()}")
print(f"   Shape: {df_clean.shape}\n")

#### 8. `enrolled_university`
**Action:** Fill missing with 'no_enrollment', One-Hot encoding  
**Reason:** 2% missing; 3 nominal categories (enrollment status)

In [ ]:
# Feature 8: enrolled_university
print("="*70)
print("Processing Feature 8: enrolled_university")
print("="*70)

missing_count = df_clean['enrolled_university'].isna().sum()
print(f"\nMissing before: {missing_count} ({missing_count/len(df_clean)*100:.2f}%)")

# Step 1: Fill missing values
print("\nStep 1: Fill missing values with 'no_enrollment'")
df_clean['enrolled_university'] = df_clean['enrolled_university'].fillna('no_enrollment')
print(f"   Missing after: {df_clean['enrolled_university'].isna().sum()}")

# Step 2: One-Hot encoding
print("\nStep 2: One-Hot encoding")
enrolled_dummies = pd.get_dummies(df_clean['enrolled_university'], prefix='enrolled', drop_first=False)
df_clean = pd.concat([df_clean, enrolled_dummies], axis=1)
df_clean = df_clean.drop('enrolled_university', axis=1)

print(f"   Created {len(enrolled_dummies.columns)} binary columns:")
for col in sorted(enrolled_dummies.columns):
    count = df_clean[col].sum()
    print(f"      {col:35s}: {count:,} samples")

print(f"\nFeature 8 processed: enrolled_university")
print(f"   Shape: {df_clean.shape}\n")

#### 9. `relevent_experience`
**Action:** Binary encoding (Has→1, No→0)  
**Reason:** No missing values; binary feature with clear yes/no distinction

In [ ]:
# Feature 9: relevent_experience
print("="*70)
print("Processing Feature 9: relevent_experience")
print("="*70)

missing_count = df_clean['relevent_experience'].isna().sum()
print(f"\nMissing before: {missing_count} ({missing_count/len(df_clean)*100:.2f}%)")

# Binary encoding
print("\nStep 1: Binary encoding")
df_clean['relevent_experience'] = df_clean['relevent_experience'].map({
    'Has relevent experience': 1,
    'No relevent experience': 0
})
print(f"   Encoded: 'Has relevent experience'->1, 'No relevent experience'->0")

print(f"\nFeature 9 processed: relevent_experience")
print(f"   Type: {df_clean['relevent_experience'].dtype}")
print(f"   Distribution: {df_clean['relevent_experience'].value_counts().to_dict()}")
print(f"   Missing after: {df_clean['relevent_experience'].isna().sum()}")
print(f"   Shape: {df_clean.shape}\n")

#### 10. `last_new_job`
**Action:** Fill missing with mode, ordinal encoding ('never'→0 to '>4'→5)  
**Reason:** 2% missing; ordinal relationship (recency of job change)

In [ ]:
# Feature 10: last_new_job
print("="*70)
print("Processing Feature 10: last_new_job")
print("="*70)

missing_count = df_clean['last_new_job'].isna().sum()
print(f"\nMissing before: {missing_count} ({missing_count/len(df_clean)*100:.2f}%)")

# Step 1: Fill missing with mode
print("\nStep 1: Fill missing with mode")
mode_value = df_clean['last_new_job'].mode()[0]
df_clean['last_new_job'] = df_clean['last_new_job'].fillna(mode_value)
print(f"   Filled {missing_count:,} missing values with mode '{mode_value}'")

# Step 2: Ordinal encoding
print("\nStep 2: Ordinal encoding (job change recency)")
job_mapping = {'never': 0, '1': 1, '2': 2, '3': 3, '4': 4, '>4': 5}
df_clean['last_new_job'] = df_clean['last_new_job'].map(job_mapping)
print(f"   Encoded: 'never'->0, '1'->1, '2'->2, '3'->3, '4'->4, '>4'->5")

print(f"\nFeature 10 processed: last_new_job")
print(f"   Type: {df_clean['last_new_job'].dtype}, Range: [{df_clean['last_new_job'].min()}, {df_clean['last_new_job'].max()}]")
print(f"   Missing after: {df_clean['last_new_job'].isna().sum()}")
print(f"   Shape: {df_clean.shape}\n")

#### 11. `city`
**Action:** Target encoding after dataset split (to prevent data leakage)  
**Reason:** High cardinality (123 unique values); avoid curse of dimensionality

In [ ]:
# Feature 11: city
print("="*70)
print("Processing Feature 11: city")
print("="*70)

missing_count = df_clean['city'].isna().sum()
print(f"\nMissing before: {missing_count} ({missing_count/len(df_clean)*100:.2f}%)")
print(f"Unique cities: {df_clean['city'].nunique()}")

print("\nNote: Target encoding will be applied AFTER dataset split")
print("   Reason: Prevent data leakage (target info must not leak from train to test)")
print("   High cardinality (123 cities) - unsuitable for One-Hot encoding")

print(f"\nFeature 11 prepared: city (encoding deferred)")
print(f"   Will encode after train/validation/test split")
print(f"   Shape: {df_clean.shape}\n")

#### 12-13. `city_development_index` & `training_hours`
**Action:** Keep as-is (no transformation needed)  
**Reason:** Already numeric, no missing values, ready for modeling

In [ ]:
# Features 12-13: city_development_index & training_hours
print("="*70)
print("Processing Features 12-13: Numeric Features")
print("="*70)

# Feature 12: city_development_index
missing_count_cdi = df_clean['city_development_index'].isna().sum()
print(f"\nFeature 12: city_development_index")
print(f"   Missing: {missing_count_cdi} ({missing_count_cdi/len(df_clean)*100:.2f}%)")
print(f"   Type: {df_clean['city_development_index'].dtype}")
print(f"   Range: [{df_clean['city_development_index'].min():.3f}, {df_clean['city_development_index'].max():.3f}]")
print(f"   Already numeric, no transformation needed")

# Feature 13: training_hours
missing_count_th = df_clean['training_hours'].isna().sum()
print(f"\nFeature 13: training_hours")
print(f"   Missing: {missing_count_th} ({missing_count_th/len(df_clean)*100:.2f}%)")
print(f"   Type: {df_clean['training_hours'].dtype}")
print(f"   Range: [{df_clean['training_hours'].min()}, {df_clean['training_hours'].max()}] hours")
print(f"   Already numeric, no transformation needed")

print(f"\nFeatures 12-13 processed: city_development_index & training_hours")
print(f"   Shape: {df_clean.shape}\n")

In [ ]:
# Verification: Check cleaned data quality
print("="*80)
print("DATA CLEANING VERIFICATION")
print("="*80)

# 1. Shape comparison
print("\n1. Dataset Shape:")
print(f"   Original: {df.shape}")
print(f"   Cleaned:  {df_clean.shape}")
print(f"   Rows preserved: {df_clean.shape[0]} (100%)")
print(f"   Columns changed: {df.shape[1]} → {df_clean.shape[1]} (due to One-Hot encoding)")

# 2. Missing values check
print("\n2. Missing Values:")
missing_after = df_clean.isnull().sum().sum()
if missing_after == 0:
    print(f"   No missing values! All {df_clean.shape[0] * df_clean.shape[1]:,} cells are filled")
else:
    print(f"   WARNING: {missing_after} missing values found!")
    print(df_clean.isnull().sum()[df_clean.isnull().sum() > 0])

# 3. Data types check
print("\n3. Data Types:")
dtypes_summary = df_clean.dtypes.value_counts()
print(f"   {dtypes_summary.to_dict()}")
non_numeric = df_clean.select_dtypes(include=['object']).columns.tolist()
if len(non_numeric) == 0:
    print(f"   All features are numeric (ready for modeling)")
else:
    print(f"   Non-numeric columns found: {non_numeric}")

# 4. Feature list
print("\n4. Final Feature List:")
feature_cols = [col for col in df_clean.columns if col != 'target']
print(f"   Total features: {len(feature_cols)}")
print(f"   Features: {', '.join(sorted(feature_cols)[:10])}...")

# 5. Display sample
print("\n5. Sample of Cleaned Data (first 3 rows):")
print("="*80)
display(df_clean.head(3))

print("\n" + "="*80)
print("DATA CLEANING COMPLETED SUCCESSFULLY!")
print("="*80)

## 4. Dataset Split & Target Encoding (5-Fold Cross-Validation Strategy)

**Overview:**
This section implements a 5-Fold Stratified Cross-Validation strategy to maximize data utilization and improve model evaluation reliability.

**Key Steps:**
1. **Separate test set (20%)** - Hold out for final evaluation
2. **Setup 5-Fold CV on remaining 80%** - For model training and validation
3. **Target encode `city` feature** - Handle high cardinality without data leakage
4. **Verify data quality** - Ensure all features are numeric and ready


### 4.1 Prepare Features and Target Variable

In [ ]:
from sklearn.model_selection import train_test_split, StratifiedKFold
import numpy as np

# Separate features and target variable
X = df_clean.drop('target', axis=1)
y = df_clean['target'].astype(int)

print("="*80)
print("Dataset Preparation")
print("="*80)
print(f"Feature matrix X: {X.shape}")
print(f"Target variable y: {y.shape}")
print(f"\nTarget distribution:")
print(f"  Class 0 (Not looking for change): {(y==0).sum():,} ({(y==0).sum()/len(y)*100:.2f}%)")
print(f"  Class 1 (Looking for change):     {(y==1).sum():,} ({(y==1).sum()/len(y)*100:.2f}%)")
print(f"  Class ratio: {(y==0).sum()/(y==1).sum():.2f}:1")

### 4.2 Dataset Split Strategy: 5-Fold Stratified Cross-Validation

**Why 5-Fold CV?**
- **Better data utilization**: 80% used for training (vs 60% in 3-way split)
- **More reliable evaluation**: Each sample serves as validation once, averaged over 5 folds
- **Reduced variance**: Multiple validation results -> more stable performance estimate

**Split Strategy:**
1. **Test set (20%)**: Hold out completely, only used for final evaluation
2. **Train+Validation (80%)**: Split into 5 folds for cross-validation
   - Each fold: 4 parts training (64%) + 1 part validation (16%)
   - Repeat 5 times, every sample gets validated once

In [ ]:
# Step 1: Separate test set (20%)
X_train_val, X_test, y_train_val, y_test = train_test_split(
    X, y,
    test_size=0.20,           # 20% for test set
    stratify=y,               # Stratified sampling to preserve class ratio
    random_state=22207256           # Fixed seed for reproducibility
)

print("="*80)
print("Step 1: Test Set Separation")
print("="*80)
print(f"Train+Validation set: {X_train_val.shape[0]:,} samples ({X_train_val.shape[0]/len(X)*100:.1f}%)")
print(f"Test set:             {X_test.shape[0]:,} samples ({X_test.shape[0]/len(X)*100:.1f}%)")

# Verify class distribution in test set
print(f"\nTest set class distribution:")
print(f"  Class 0: {(y_test==0).sum():,} ({(y_test==0).sum()/len(y_test)*100:.2f}%)")
print(f"  Class 1: {(y_test==1).sum():,} ({(y_test==1).sum()/len(y_test)*100:.2f}%)")
print(f"  Ratio: {(y_test==0).sum()/(y_test==1).sum():.2f}:1")

# Step 2: Setup 5-Fold Stratified Cross-Validation on Train+Val set
print("\n" + "="*80)
print("Step 2: Setup 5-Fold Stratified Cross-Validation")
print("="*80)

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=22307256)

print(f"Configuration:")
print(f"  Number of folds: 5")
print(f"  Total samples for CV: {X_train_val.shape[0]:,}")
print(f"  Samples per fold (validation): ~{X_train_val.shape[0]//5:,} ({100/5:.1f}%)")
print(f"  Samples per fold (training): ~{X_train_val.shape[0]*4//5:,} ({100*4/5:.1f}%)")

# Verify fold quality
print(f"\n" + "="*80)
print("Fold Quality Verification:")
print("="*80)
print(f"\n{'Fold':<8} {'Train Samples':<15} {'Val Samples':<15} {'Class 0 (Val)':<15} {'Class 1 (Val)':<15} {'Ratio':<10}")
print("-" * 90)

for fold, (train_idx, val_idx) in enumerate(skf.split(X_train_val, y_train_val), 1):
    y_val_fold = y_train_val.iloc[val_idx]
    class_0 = (y_val_fold == 0).sum()
    class_1 = (y_val_fold == 1).sum()
    ratio = class_0 / class_1 if class_1 > 0 else 0
    
    print(f"Fold {fold:<3} {len(train_idx):<15,} {len(val_idx):<15,} {class_0:<7,} ({class_0/len(y_val_fold)*100:5.2f}%)  {class_1:<7,} ({class_1/len(y_val_fold)*100:5.2f}%)  {ratio:.2f}:1")

print("\n" + "="*80)
print("Dataset Split Summary")
print("="*80)
print(f"Train+Validation: {X_train_val.shape[0]:,} samples ({X_train_val.shape[0]/len(X)*100:.1f}%) - Used for 5-Fold CV")
print(f"Test set:         {X_test.shape[0]:,} samples ({X_test.shape[0]/len(X)*100:.1f}%) - Held out for final evaluation")
print(f"Total:            {len(X):,} samples (100%)")

### 4.3 Target Encoding for `city`

**Why Target Encoding?**
- `city` has 123 unique values (high cardinality)
- One-Hot encoding would create 123 columns (curse of dimensionality)
- Target encoding maps each city to its average target value

**Preventing Data Leakage:**
- Encoder fitted ONLY on training data
- Then applied to validation and test sets
- Cross-validation built into TargetEncoder (cv=5)

In [ ]:
from sklearn.preprocessing import TargetEncoder

print("="*80)
print("Feature 11: city")
print("="*80)

# Initialize Target Encoder
target_encoder = TargetEncoder(
    cv=5,                    # 5-fold cross-validation to prevent overfitting
    smooth='auto',           # Automatic smoothing for low-frequency cities
    target_type='binary'     # Binary classification target
)

# Step 1: Fit encoder on Train+Validation set
print("\nStep 1: Fit encoder on Train+Validation set")
print(f"   Unique cities: {X_train_val['city'].nunique()}")
print(f"   Total samples: {len(X_train_val):,}")

target_encoder.fit(X_train_val[['city']], y_train_val)
print("   Encoder trained successfully")

# Step 2: Transform Train+Validation set
print("\nStep 2: Encode Train+Validation set")
X_train_val_encoded = X_train_val.copy()
X_train_val_encoded['city_encoded'] = target_encoder.transform(X_train_val[['city']])
X_train_val_encoded = X_train_val_encoded.drop('city', axis=1)
print(f"   Train+Val set encoded")
print(f"   Encoding range: [{X_train_val_encoded['city_encoded'].min():.4f}, {X_train_val_encoded['city_encoded'].max():.4f}]")

# Step 3: Transform Test set
print("\nStep 3: Encode Test set (using Train+Val encoding)")
X_test_encoded = X_test.copy()
X_test_encoded['city_encoded'] = target_encoder.transform(X_test[['city']])
X_test_encoded = X_test_encoded.drop('city', axis=1)
print(f"   Test set encoded")
print(f"   Test set cities: {X_test['city'].nunique()}")

print("\n" + "="*80)
print("Target Encoding Complete")
print("="*80)
print(f"Train+Validation set: {X_train_val_encoded.shape}")
print(f"Test set:             {X_test_encoded.shape}")
print(f"\nAll features are now numeric and ready for 5-Fold Cross-Validation!")

### 4.4 Final Dataset Summary

In [ ]:
# Final dataset summary
print("="*80)
print("Final Dataset Summary")
print("="*80)

print("\n1. Dataset Shapes:")
print(f"   Train+Validation: X_train_val_encoded {X_train_val_encoded.shape}, y_train_val {y_train_val.shape}")
print(f"   Test set:         X_test_encoded      {X_test_encoded.shape}, y_test      {y_test.shape}")

print("\n2. Number of Features:")
print(f"   Total features: {X_train_val_encoded.shape[1]}")

print("\n3. Missing Values Check:")
print(f"   Train+Val missing: {X_train_val_encoded.isnull().sum().sum()}")
print(f"   Test missing:      {X_test_encoded.isnull().sum().sum()}")

print("\n4. Feature List (all features):")
feature_cols = X_train_val_encoded.columns.tolist()
for i, col in enumerate(feature_cols, 1):
    print(f"   {i:2d}. {col}")

print("\n5. Sample Data (Train+Val first 3 rows):")
display(X_train_val_encoded.head(3))

print("\n" + "="*80)
print("Dataset Preparation Complete!")
print("="*80)
print(f"Ready for 5-Fold Cross-Validation with {X_train_val_encoded.shape[0]:,} samples")
print(f"Test set ({X_test_encoded.shape[0]:,} samples) reserved for final evaluation")